<a href="https://colab.research.google.com/github/deiveleal/multi_modal_machine_learning/blob/main/PrimeiroModelo/MultiModalAlgorithm-ExtracFeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Multimodal Algorithm

Data &#8594; Pre-process &#8594; Separate Training &#8594; Fusion &#8594; Metric

### Hello world (Iris)

In [21]:
# import os

# def rename_files_in_directory(directory_path):
#     """
#     Renomeia todos os arquivos em um diretório para o nome da pasta seguido de um número crescente.
#     Substitui "-" por "_" no nome da pasta.

#     Args:
#         directory_path: Caminho para o diretório.
#     """
#     # Obtém o nome da pasta e substitui "-" por "_"
#     folder_name = os.path.basename(directory_path).replace("-", "_")
#     # Lista todos os arquivos no diretório
#     filenames = os.listdir(directory_path)

#     # Para cada arquivo no diretório
#     for i, filename in enumerate(filenames):
#         # Constrói o novo nome do arquivo
#         new_filename = f"{folder_name}_{i+1}.jpg"
#         # Constrói o caminho completo para o arquivo antigo e o novo
#         old_file_path = os.path.join(directory_path, filename)
#         new_file_path = os.path.join(directory_path, new_filename)
#         # Renomeia o arquivo
#         os.rename(old_file_path, new_file_path)

# # Uso da função
# rename_files_in_directory('sample/iris_raw_images/iris-setosa')
# rename_files_in_directory('sample/iris_raw_images/iris-versicolor')
# rename_files_in_directory('sample/iris_raw_images/iris-virginica')

In [25]:
# import shutil
# import glob

# # Defina o caminho do arquivo de origem e o caminho do destino
# src_dir = 'sample/iris_raw_images/iris-versicolor/*'
# dst_dir = 'sample/iris_images/'

# # Use glob para obter todos os arquivos no diretório de origem
# files = glob.glob(src_dir)

# # Itere sobre a lista de arquivos e copie cada um para o novo diretório
# for file in files:
#     shutil.copy(file, dst_dir)

### Processa imagem (cria um vetor com a imagem)

In [2]:
import os

import numpy as np
from sklearn import preprocessing
from sklearn.cluster import KMeans
from PIL import Image
import cv2
import pandas as pd

# Alterar as opções de impressão
# np.set_printoptions(threshold=np.inf)

In [2]:
def vetorizar_imagem(caminho_imagem):
    # Ler a imagem
    imagem = cv2.imread(caminho_imagem)

    # Converter a imagem em um vetor unidimensional
    vetor_imagem = imagem.flatten()

    return vetor_imagem

In [3]:
def vetorizar_imagens(diretorio):
    vetor_imagens = {}
    for root, dirs, files in os.walk(diretorio):
        for filename in files:
            if filename.endswith(".jpg") or filename.endswith(".png"):
                caminho_imagem = os.path.join(root, filename)
                imagem = cv2.imread(caminho_imagem, cv2.IMREAD_GRAYSCALE)
                vetor_imagem = imagem.flatten()
                file_name = filename.split('.')[0]
                vetor_imagens[file_name] = np.array(vetor_imagem)
    return vetor_imagens

In [4]:
# Use a função para vetorizar todas as imagens em um diretório
# images_vectorized = {}
directory = 'sample/iris/test'
# images_vectorized = vetorizar_imagens(directory)
# Exemplo de uso
vetor_imagens = vetorizar_imagens(directory)

In [5]:
vetor_imagens

{'iris_versicolor_269': array([66, 69, 73, ...,  8,  8,  8], dtype=uint8),
 'iris_setosa_65': array([184, 183, 185, ..., 153, 134, 157], dtype=uint8),
 'iris_setosa_66': array([209, 131,  89, ...,  76,  97,  53], dtype=uint8),
 'iris_virginica_71': array([86, 86, 87, ..., 80, 79, 78], dtype=uint8),
 'iris_versicolor_233': array([ 76,  79,  82, ..., 124, 126, 128], dtype=uint8),
 'iris_versicolor_223': array([ 5,  8,  4, ..., 38, 39, 39], dtype=uint8),
 'iris_versicolor_240': array([83, 47, 37, ..., 68, 67, 88], dtype=uint8),
 'iris_versicolor_261': array([130, 131, 133, ...,  96,  85,  87], dtype=uint8),
 'iris_virginica_81': array([ 14,  14,  14, ..., 148, 143, 136], dtype=uint8),
 'iris_versicolor_256': array([ 88,  88,  86, ..., 126, 133, 137], dtype=uint8),
 'iris_versicolor_236': array([69, 54, 46, ..., 82, 85, 86], dtype=uint8),
 'iris_setosa_67': array([140,  90,  58, ...,  99,  45,  45], dtype=uint8),
 'iris_versicolor_260': array([ 3,  0, 18, ..., 22,  2,  1], dtype=uint8),
 '

In [26]:
def extrair_cor_petalas(caminho_imagem):
    # Carregar a imagem
    imagem = Image.open(caminho_imagem)
    imagem = imagem.resize((150,150)) # Redimensionar a imagem para reduzir o tempo de processamento
    imagem_np = np.array(imagem)

    # Redimensionar a imagem para ser uma lista de pixels
    pixels = imagem_np.reshape(-1, 3)

    # Executar o algoritmo k-means para encontrar as cores mais comuns
    kmeans = KMeans(n_clusters=5)
    kmeans.fit(pixels)

    # Encontrar a cor mais comum (ou seja, o centro do cluster mais comum)
    cor_dominante = kmeans.cluster_centers_[kmeans.labels_[0]]

    return cor_dominante

In [27]:
def extrair_cores_pasta(caminho_pasta):
    # Inicializar um dicionário para armazenar as cores dominantes
    cores_dominantes = {}

    # Listar todos os arquivos na pasta
    arquivos = os.listdir(caminho_pasta)

    # Para cada arquivo na pasta, extrair a cor dominante das pétalas
    for arquivo in arquivos:
        # Ignorar arquivos que não são imagens
        if not (arquivo.endswith('.jpg') or arquivo.endswith('.png')):
            continue

        # Construir o caminho completo para o arquivo
        caminho_arquivo = os.path.join(caminho_pasta, arquivo)

        # Extrair a cor dominante das pétalas
        cor_dominante = extrair_cor_petalas(caminho_arquivo)

        # Adicionar a cor dominante ao dicionário
        cores_dominantes[arquivo.split('.')[0]] = cor_dominante

    return cores_dominantes


In [28]:
# Exemplo de uso
cores_dominantes = extrair_cores_pasta('sample/iris_images')
for nome_arquivo, cor_dominante in cores_dominantes.items():
    print("Cor dominante das pétalas do arquivo '{}': {}".format(nome_arquivo, cor_dominante))

Cor dominante das pétalas do arquivo 'iris_setosa_27': [27.6259516  35.86512896 24.74434723]
Cor dominante das pétalas do arquivo 'iris_versicolor_144': [40.68561942 40.53787093 25.12948745]
Cor dominante das pétalas do arquivo 'iris_versicolor_269': [58.74229346 74.80049322 38.71270037]
Cor dominante das pétalas do arquivo 'iris_versicolor_129': [42.59793137 56.75340667 38.29945822]
Cor dominante das pétalas do arquivo 'iris_versicolor_175': [116.80502363 125.43906988  48.84730167]
Cor dominante das pétalas do arquivo 'iris_virginica_3': [48.10423656 94.17234922 10.75314036]
Cor dominante das pétalas do arquivo 'iris_versicolor_199': [230.9600779  238.60662123 240.80623174]
Cor dominante das pétalas do arquivo 'iris_versicolor_133': [48.62858486 60.72903934 33.4933396 ]
Cor dominante das pétalas do arquivo 'iris_virginica_49': [154.5627055  191.68529826 117.20056364]
Cor dominante das pétalas do arquivo 'iris_versicolor_83': [190.10673135 183.88720437 202.05882353]
Cor dominante das p

In [29]:
def extrair_formato_petalas(caminho_imagem):
    # Carregar a imagem em escala de cinza
    imagem = cv2.imread(caminho_imagem, cv2.IMREAD_GRAYSCALE)

    # Aplicar um limiar para separar as pétalas do fundo
    _, imagem_limiar = cv2.threshold(imagem, 127, 255, cv2.THRESH_BINARY)

    # Encontrar os contornos na imagem
    contornos, _ = cv2.findContours(imagem_limiar, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Calcular a área e o perímetro de cada contorno
    areas = [cv2.contourArea(contorno) for contorno in contornos]
    perimetros = [cv2.arcLength(contorno, True) for contorno in contornos]

    # Calcular a convexidade de cada contorno (a proporção entre a área do contorno e a área do seu casco convexo)
    convexidades = [cv2.contourArea(contorno) / cv2.contourArea(cv2.convexHull(contorno)) if cv2.contourArea(cv2.convexHull(contorno)) != 0 else 0 for contorno in contornos]

    return areas, perimetros, convexidades

In [30]:
def extrair_caracteristicas_pasta(caminho_pasta):
    # Obter uma lista de todos os arquivos na pasta
    arquivos = os.listdir(caminho_pasta)

    # Inicializar um dicionário para armazenar as características de cada arquivo
    caracteristicas_arquivos = {}

    # Para cada arquivo na pasta, extrair as características e adicionar ao dicionário
    for arquivo in arquivos:
        # Ignorar arquivos que não são imagens
        if not (arquivo.endswith('.png') or arquivo.endswith('.jpg')):
            continue

        # Extrair as características do arquivo
        areas, perimetros, convexidades = extrair_formato_petalas(os.path.join(caminho_pasta, arquivo))

        # Adicionar as características ao dicionário, usando o nome do arquivo sem a extensão como chave
        nome_arquivo_sem_extensao = os.path.splitext(arquivo)[0]
        caracteristicas_arquivos[nome_arquivo_sem_extensao] = {"areas": areas, "perimetros":perimetros, "convexidades":convexidades}
    # Extrair o formato das pétalas

    return caracteristicas_arquivos

In [31]:
# # Exemplo de uso
# caracteristicas = extrair_caracteristicas('sample/iris/iris_setosa_1.jpg')

# print("Características da imagem:", caracteristicas)


# Exemplo de uso
caracteristicas_iris = extrair_caracteristicas_pasta('sample/iris_images')

for nome_arquivo, caracteristicas in caracteristicas_iris.items():
    print("Características do arquivo iris'{}': {}".format(nome_arquivo, caracteristicas))

Características do arquivo iris'iris_setosa_27': {'areas': [6.5, 0.0, 12.5, 2.5, 40.5, 0.0, 11.0, 0.0, 0.0, 0.0, 32.5, 0.0, 0.0, 20.0, 0.0, 4.0, 0.0, 2.5, 0.0, 0.0, 68.5, 26.0, 0.5, 0.0, 0.0, 0.0, 29.0, 298.5, 93.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 14.5, 273.5, 1.0, 0.0, 49.5, 13.5, 15.5, 0.0, 2.0, 0.0, 47.0, 5.0, 26.5, 41.5, 3.0, 27.5, 0.0, 0.5, 0.0, 1.5, 11.0, 7.0, 54.0, 0.0, 6.0, 2.0, 0.0, 292.5, 0.0, 1.0, 0.0, 0.0, 0.0, 4.5, 22.5, 1.0, 9.5, 1.0, 51.5, 2.0, 34.0, 0.0, 145.5, 0.0, 9.0, 5.5, 0.0, 2.0, 0.0, 51.5, 5.0, 954.0, 0.0, 426.5, 0.0, 0.0, 0.5, 11.5, 0.0, 48.0, 0.0, 0.0, 34.0, 0.0, 0.5, 0.0, 19.5, 0.0, 201.5, 0.0, 38.0, 1133.5, 10.0, 30.0, 0.0, 1.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 6026.5, 899.5], 'perimetros': [15.071067690849304, 2.0, 17.071067690849304, 7.414213538169861, 87.98275470733643, 4.0, 15.656854152679443, 7.656854152679443, 4.0, 2.8284270763397217, 36.72792184352875, 0.0, 8.0, 18.485281229019165, 2

In [32]:
def reduzir_dimensao(dicionario_caracteristicas):
    dicionario_reduzido = {}

    for nome_arquivo, caracteristicas in dicionario_caracteristicas.items():
        caracteristicas_reduzidas = {chave: [np.mean(valores)] for chave, valores in caracteristicas.items()}
        dicionario_reduzido[nome_arquivo] = caracteristicas_reduzidas

    return dicionario_reduzido

caracteristicas_iris_reduzidas = reduzir_dimensao(caracteristicas_iris)

for nome_arquivo, caracteristicas in caracteristicas_iris_reduzidas.items():
    print("Características reduzidas do arquivo iris'{}': {}".format(nome_arquivo, caracteristicas))

Características reduzidas do arquivo iris'iris_setosa_27': {'areas': [87.53731343283582], 'perimetros': [39.288602047002136], 'convexidades': [0.3452148033357001]}
Características reduzidas do arquivo iris'iris_versicolor_144': {'areas': [162.875], 'perimetros': [49.677266976412604], 'convexidades': [0.3464588300754967]}
Características reduzidas do arquivo iris'iris_versicolor_269': {'areas': [189.98717948717947], 'perimetros': [34.18008995973147], 'convexidades': [0.26005680607783016]}
Características reduzidas do arquivo iris'iris_versicolor_129': {'areas': [16.974093264248705], 'perimetros': [14.786588145043567], 'convexidades': [0.29998046219166913]}
Características reduzidas do arquivo iris'iris_versicolor_175': {'areas': [224.0], 'perimetros': [57.85290170932303], 'convexidades': [0.3369053339911783]}
Características reduzidas do arquivo iris'iris_virginica_3': {'areas': [96.99509803921569], 'perimetros': [28.688537077576505], 'convexidades': [0.22386095929134228]}
Característic

In [33]:
cores_dominantes_df = pd.DataFrame(cores_dominantes).T
cores_dominantes_df.columns = ['cor_r', 'cor_g', 'cor_b']
cores_dominantes_df = cores_dominantes_df.reset_index().rename(columns={'index':'arquivo_cor'})
cores_dominantes_df

,arquivo_cor,cor_r,cor_g,cor_b
0,iris_setosa_27,27.625952,35.865129,24.744347
1,iris_versicolor_144,40.685619,40.537871,25.129487
2,iris_versicolor_269,58.742293,74.800493,38.712700
3,iris_versicolor_129,42.597931,56.753407,38.299458
4,iris_versicolor_175,116.805024,125.439070,48.847302
...,...,...,...,...
416,iris_versicolor_215,12.077333,12.310667,12.420000
417,iris_virginica_38,117.308557,84.051159,44.755907
418,iris_versicolor_30,59.732630,94.375644,12.241836
419,iris_virginica_60,138.977096,182.453216,89.623782


In [34]:
df = pd.DataFrame(caracteristicas_iris_reduzidas).T
df = df.reset_index().rename(columns={'index': 'nome_arquivo'})
df['classe'] = [classe.split('_')[1] for classe in df['nome_arquivo']]
df['areas'] = [area[0] for area in df['areas']]
df['perimetros'] = [perimetro[0] for perimetro in df['perimetros']]
df['convexidades'] = [convexidade[0] for convexidade in df['convexidades']]   
df

,nome_arquivo,areas,perimetros,convexidades,classe
0,iris_setosa_27,87.537313,39.288602,0.345215,setosa
1,iris_versicolor_144,162.875000,49.677267,0.346459,versicolor
2,iris_versicolor_269,189.987179,34.180090,0.260057,versicolor
3,iris_versicolor_129,16.974093,14.786588,0.299980,versicolor
4,iris_versicolor_175,224.000000,57.852902,0.336905,versicolor
...,...,...,...,...,...
416,iris_versicolor_215,1578.892857,165.469268,0.350179,versicolor
417,iris_virginica_38,609.389706,54.003962,0.202788,virginica
418,iris_versicolor_30,468.067568,78.600446,0.321060,versicolor
419,iris_virginica_60,176.313559,53.603482,0.340828,virginica


In [35]:
df_carac = pd.merge(df, cores_dominantes_df, left_on='nome_arquivo', right_on='arquivo_cor')
df_carac.drop(columns=['arquivo_cor'], inplace=True)
df_carac

,nome_arquivo,areas,perimetros,convexidades,classe,cor_r,cor_g,cor_b
0,iris_setosa_27,87.537313,39.288602,0.345215,setosa,27.625952,35.865129,24.744347
1,iris_versicolor_144,162.875000,49.677267,0.346459,versicolor,40.685619,40.537871,25.129487
2,iris_versicolor_269,189.987179,34.180090,0.260057,versicolor,58.742293,74.800493,38.712700
3,iris_versicolor_129,16.974093,14.786588,0.299980,versicolor,42.597931,56.753407,38.299458
4,iris_versicolor_175,224.000000,57.852902,0.336905,versicolor,116.805024,125.439070,48.847302
...,...,...,...,...,...,...,...,...
416,iris_versicolor_215,1578.892857,165.469268,0.350179,versicolor,12.077333,12.310667,12.420000
417,iris_virginica_38,609.389706,54.003962,0.202788,virginica,117.308557,84.051159,44.755907
418,iris_versicolor_30,468.067568,78.600446,0.321060,versicolor,59.732630,94.375644,12.241836
419,iris_virginica_60,176.313559,53.603482,0.340828,virginica,138.977096,182.453216,89.623782


In [ ]:
def normalizar_colunas(df):
    return df.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [ ]:
df_normalizado = normalizar_colunas(df_carac[['areas','perimetros','convexidades','cor_r','cor_g','cor_b']])
print(df_normalizado)
df_carac[['areas','perimetros','convexidades','cor_r','cor_g','cor_b']] = df_normalizado[['areas','perimetros','convexidades','cor_r','cor_g','cor_b']]
df_carac

In [36]:
# df_carac['geral'] = [np.array([df_carac['areas'][i], df_carac['perimetros'][i], df_carac['convexidades'][i], df_carac['cor_r'][i], df_carac['cor_g'][i], df_carac['cor_b'][i]]) for i in range(len(df_carac))]
# df_carac['areas'][2]
df_carac.sort_values(by='nome_arquivo', inplace=True)
df_carac.reset_index(drop=True, inplace=True)
df_carac

,nome_arquivo,areas,perimetros,convexidades,classe,cor_r,cor_g,cor_b
0,iris_setosa_1,128.996599,40.107930,0.208536,setosa,126.535235,147.174287,78.032657
1,iris_setosa_10,26.356037,18.948713,0.272922,setosa,125.263693,130.347865,126.853705
2,iris_setosa_11,62.977564,29.898656,0.324484,setosa,111.139645,176.319527,103.782544
3,iris_setosa_12,96.663462,41.234229,0.212876,setosa,49.511023,115.586029,8.272346
4,iris_setosa_13,116.768817,44.329692,0.279369,setosa,63.774974,112.011813,26.527490
...,...,...,...,...,...,...,...,...
416,iris_virginica_82,4786.269231,126.481302,0.109638,virginica,177.551741,194.588277,155.137509
417,iris_virginica_83,313.449495,53.691366,0.307275,virginica,116.343071,129.832509,28.588614
418,iris_virginica_84,44.822981,25.492963,0.218105,virginica,115.827655,132.540412,71.038431
419,iris_virginica_85,49.551282,28.305283,0.478177,virginica,62.562697,92.157837,23.097934


In [37]:
df_carac.to_csv('sample/iris_images.csv', index=False, sep=',', header=True)

In [ ]:
def early_fusion(caracteristicas, caracteristicas_outras):
    # Inicializar um dicionário para armazenar os vetores fusionados
    dicionario_fusion = {}

    # Para cada arquivo no dicionário de características, adicionar o vetor fusionado ao dicionario_fusion
    for nome_arquivo, caracteristica in caracteristicas.items():
        # Ignorar arquivos que não estão no dicionario_cores
        if nome_arquivo not in caracteristicas_outras:
            continue

        # Obter a cor correspondente
        outra = caracteristicas_outras[nome_arquivo]

        # Realizar a fusão precoce concatenando as características e a cor
        vetor_fusion = caracteristica + outra

        # Adicionar o vetor fusionado ao dicionario_fusion
        dicionario_fusion[nome_arquivo] = vetor_fusion

    return dicionario_fusion

In [ ]:
# for nome_arquivo, caracteristica in caracteristicas_iris_reduzidas.items():
    # print("Características reduzidas do arquivo iris'{}': {}".format(nome_arquivo, caracteristicas))
for nome_arquivo, caracteristica in caracteristicas_iris_reduzidas.items():
    nova_caracteristica = []
    nova_caracteristica.append(caracteristica['areas'][0])
    nova_caracteristica.append(caracteristica['perimetros'][0])
    nova_caracteristica.append(caracteristica['convexidades'][0])
    caracteristica_iris = {nome_arquivo: np.array(nova_caracteristica)}
    print("Características early fusion do arquivo iris'{}': {}".format(nome_arquivo, caracteristica_iris[nome_arquivo]))

In [ ]:
caracteristica_iris

In [ ]:
fusion_areas_diametros = early_fusion(caracteristicas_iris_reduzidas, images_vectorized)

In [ ]:
caracteristica_iris_cores = {}
for nome_arquivo_cor, caracteristica_cor in cores_dominantes.items():
    for nome_arquivo, caracteristica in caracteristica_iris.items():
        if nome_arquivo_cor == nome_arquivo:
            print(True)
            array_caracteristica = np.append(caracteristica, caracteristica_cor)
            caracteristica_iris_cores[nome_arquivo] = array_caracteristica

In [ ]:
caracteristica_iris_cores

In [ ]:
for nome_arquivo_cor, caracteristica_cor in cores_dominantes.items(): 
    print(nome_arquivo_cor, caracteristica_cor)

In [ ]:
caracteristica_iris_cores

In [ ]:
cores_dominantes

### Gemini Tutorial

In [ ]:
import pandas as pd

# Carregar o dataset
data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00456/Multimodal+Damage+Identification+for+Humanitarian+Computing.csv")

# Separar as features em imagens, textos e informações tabulares
images = data["image_url"]
texts = data["text"]
tabular_data = data.drop(["image_url", "text"], axis=1)


In [ ]:
# Carregar imagens
images = tf.keras.utils.image_dataset_from_directory(
    directory="data/images",
    label_mode="categorical",
    image_size=(224, 224),
    batch_size=32
)

# Carregar áudio
audio = tf.keras.utils.audio_dataset_from_directory(
    directory="data/audio",
    label_mode="categorical",
    batch_size=32
)


In [17]:
# import os
# import glob

# # Defina a pasta e a extensão do arquivo
# folder = 'sample/iris_raw_images/iris-virginica'
# file_ext = '*.Identifier'

# # Use glob para combinar o caminho da pasta com a extensão do arquivo
# files = glob.glob(os.path.join(folder, file_ext))

# # Itere sobre a lista de arquivos e remova cada um
# for file in files:
#     os.remove(file)